In [1]:
import numpy as np,torch

from translation_machine import dataset_mod,sentence_mod

whole_dataset = dataset_mod.DatasetFromTxt("../data/french_english_dataset/fra.txt")
idxs = np.load("../dataset_splitting/idx_train.npy")
    

dataset_test = dataset_mod.SentenceDataSet(torch.utils.data.Subset(whole_dataset,idxs),
                                                   sentence_mod.EnglishSentence,
                                                   sentence_mod.FrenchSentence)


/home/amine_ammor_91/miniconda3/envs/trans_mach_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/amine_ammor_91/miniconda3/envs/trans_mach_env/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
/home/amine_ammor_91/miniconda3/envs/trans_mach_env/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "fr" could not be loaded, trying "fr_core_news_sm" instead
  warnings.warn(


In [2]:
from translation_machine import model_mod,model_bidirectionnal_mod


path = "../models/sequence_translator_bidirectionnal.pth"
back_up = torch.load(path)

model_inputs = back_up["model_inputs"]
sequence_translator = model_bidirectionnal_mod.SequenceTranslator(**model_inputs)


In [3]:
from pathlib import Path

sequence_translator.load_state_dict(back_up["model_params"])
sequence_translator.eval()

SequenceTranslator(
  (src_embedding_layer): Embedding(4076, 128)
  (dst_embedding_layer): Embedding(5407, 128)
  (encoder): LSTM(128, 256, batch_first=True, bidirectional=True)
  (decoder): LSTM(128, 512, batch_first=True)
  (linear_layer): Linear(in_features=512, out_features=5407, bias=True)
)

In [4]:
from translation_machine import translator_mod

translator = translator_mod.Translator(sequence_translator)

In [5]:
english_sentence,french_sentence = dataset_test[0]
english_sentence,french_sentence

(<translation_machine.sentence_mod._create_sentence_type.<locals>.Sentence at 0x7f2cd0114130>,
 <translation_machine.sentence_mod._create_sentence_type.<locals>.Sentence at 0x7f2c00400160>)

In [6]:
french_sentence.as_words,english_sentence.as_words

(['Va', '!'], ['Go', '.'])

In [7]:
from itertools import islice
preds_and_targets = [(translator(english_sentence),french_sentence) for english_sentence,french_sentence in islice(dataset_test,10)]
preds,targets = zip(*preds_and_targets)

In [8]:
for (el0,el1) in preds_and_targets:
    print(el0.as_words,el1.as_words)

['Allez', 'attraper', 'le', 'train', 'de', 'pleurer', '.'] ['Va', '!']
['mourut', '.'] ['Saute', '.']
['Bonjour', '-', 'huit', '.'] ['Bonjour', '!']
['-', 'tu', 'que', 'signifie', '.'] ['Attaquez', '!']
['-', 'toi', '.'] ["J'", 'ai', 'pigé', '!']
['Je', 'suis', 'tombé', 'sur', 'le', '.'] ['Je', 'suis', 'tombée', '.']
['Je', 'suis', '.'] ['Je', 'vais', 'bien', '.']
['-', 'toi', '.'] ["C'", 'est', 'exclu', '!']
['Nous', 'devons', 'régler', 'de', 'la', '.'] ['Nous', 'gagnâmes', '.']
['Soyez', '-vous', 'pas', 'à', 'elles', 'patron', 'sourire', '.'] ['Sois', 'juste', '!']
